In [32]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

RecursionError: maximum recursion depth exceeded while calling a Python object

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Christian\AppData\Ro

True

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv(r"D:\Projects\Github Repo\AI-Final-Project\Nusrat_Dataset.csv", encoding='ISO-8859-1')
# Clean the data
def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
        return text
    else:
        return ''

df['Tweets'] = df['Tweets'].apply(clean_text)
df.drop(df[df['Tweets'] == ''].index, inplace=True)
df.dropna(inplace=True)

In [8]:
df.groupby('Labels').describe()

Tweets         \
                  count unique   
Labels                           
atypical           1980   1965   
bipolar            2443   2434   
major depressive   2517   2515   
no                 1985   1984   
postpartum         3746   3739   
psychotic          2312   2303   

                                                                          
                                                                top freq  
Labels                                                                    
atypical                                        atypical depression    6  
bipolar                                            bipolar disorder    5  
major depressive                          major depressive disorder    3  
no                winner drumroll votes krystina jimenez congrat...    2  
postpartum                              think postpartum depression    3  
psychotic                                      psychotic depression    5

In [9]:
df['Labels'].value_counts()

Labels
postpartum          3746
major depressive    2517
bipolar             2443
psychotic           2312
no                  1985
atypical            1980
Name: count, dtype: int64

In [10]:
df_postpartum = df[df['Labels'] == 'postpartum']
df_major = df[df['Labels'] == 'major depressive']
df_bipolar = df[df['Labels'] == 'bipolar']
df_psychotic = df[df['Labels'] == 'psychotic']
df_no = df[df['Labels'] == 'no']
df_atypical = df[df['Labels'] == 'atypical']

In [11]:
df_postpartum.shape

(3746, 2)

In [12]:
df_postpartum_downsampled = df_postpartum.sample(df_atypical.shape[0])
df_postpartum_downsampled.shape

(1980, 2)

In [14]:
df_major_downsampled = df_major.sample(df_atypical.shape[0])
df_major_downsampled.shape

(1980, 2)

In [15]:
df_bipolar_downsampled = df_bipolar.sample(df_atypical.shape[0])
df_bipolar_downsampled.shape

(1980, 2)

In [16]:
df_psychotic_downsampled = df_psychotic.sample(df_atypical.shape[0])
df_psychotic_downsampled.shape

(1980, 2)

In [17]:
df_no_downsampled = df_no.sample(df_atypical.shape[0])
df_no_downsampled.shape

(1980, 2)

In [18]:
df_balanced = pd.concat([df_postpartum_downsampled, df_major_downsampled, df_bipolar_downsampled, df_psychotic_downsampled, df_no_downsampled, df_atypical])
df_balanced.shape

(11880, 2)

In [19]:
df_balanced['Labels'].value_counts()

Labels
postpartum          1980
major depressive    1980
bipolar             1980
psychotic           1980
no                  1980
atypical            1980
Name: count, dtype: int64

In [27]:
label_mapping = {
    'no': 0,
    'postpartum': 1,
    'major depressive': 2,
    'bipolar': 3,
    'psychotic': 4,
    'atypical': 5
}

df_balanced['Depression Type'] = df_balanced['Labels'].apply(lambda x: label_mapping[x])
df_balanced.sample(10)

,Tweets,Labels,Depression Type
7007,é½lazy sleep disorder idiopathic hypersomnia l...,atypical,5
6886,itsemilykaty cp idiopathic hypersomnia similar...,atypical,5
3902,maddierawrk feel like thread millions follower...,psychotic,4
4152,every year get little confident seasonallinked...,bipolar,3
2798,lfgandonly_ major depressive disorder every ti...,major depressive,2
4903,ssris may suitable following conditions bipola...,no,0
10307,like im stuck nightmare cant wake mind playing...,psychotic,4
14424,sadness feel like black hole like theres way e...,bipolar,3
5126,skateforsanity mandy nods é im okay love mean ...,bipolar,3
9402,yep opportunity meet polish militarily amp bus...,no,0


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Tweets'], df_balanced['Depression Type'], stratify=df_balanced['Depression Type'])
X_train.head(4)

13702    like emotions rollercoaster control one minute...
10432    feel like im losing control thoughts emotions ...
11880    youre feeling like please know youre alone sha...
7626     hypocrite exposes people amazed know whole fam...
Name: Tweets, dtype: object